# Линейная регрессия: t-тест

In [4]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_params, summary_col # вывод результатов тестирования
from scipy.stats import t # t-распределение и критические значения

# Не показывать FutureWarnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## t-тест: значимость коэффициентов

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

Тестируем нулевую гипотезу $H_0:\beta=0$

Альтернативная гипотеза $H_1:\beta\ne 0$

Тестовая статистика $$t=\frac{\hat{\beta}}{s.e.(\beta)}$$

Критическое значение $t_{cr}=t_{df}(\alpha)$, где  $df=n-k-1$ --- степени свободы для t-распределения

$\color{purple}{\text{\textbf{Значим ли коэффициент} }\beta ?}$

**<span style="color:purple">Способ 1:</span>** используем $t_{cr}$

* Отвергаем гипотезу $H_0$ при $|t|>t_{cr}$, <i><b><span style="color: purple">коэффициент значим</span></b><br>
* Не отвергаем гипотезу $H_0$ при $|t|<t_{cr}$, <i><b><span style="color: purple">коэффициент незначим</span> </b><br>

**<span style="color:purple">Способ 2:</span>** используем P-value

* Отвергаем гипотезу $H_0$  при $P<\alpha$, <i><b><span style="color: purple">коэффициент значим</span> </b><br>
* Не отвергаем гипотезу $H_0$ при $P>\alpha$, <i><b><span style="color: purple">коэффициент незначим</span> </b><br>

---

<i><b><span style="color: purple">Сравниваем самостоятельно или автоматизированно</span> <br>

<span style="color:#59afe1">ОБЯЗАТЕЛЬНО ПЕЧАТАЕМ ВЫВОД</span></b>

---

</div>

Для набора данных `sleep75` рассмотрим линейную регрессию

$$
    sleep = \beta_0+\beta_1 totwrk+\beta_2 age+\beta_3 south+\beta_4 male+\beta_5 smsa+\beta_6 yngkid+\beta_7 marr+\beta_8 union+\beta_9 \log(hrwage)+error
$$

In [5]:
# импорт данных
df = pd.read_csv('sleep75.csv')
df.head()

,age,black,case,clerical,construc,educ,earns74,gdhlth,inlf,leis1,...,spwrk75,totwrk,union,worknrm,workscnd,exper,yngkid,yrsmarr,hrwage,agesq
0,32,0,1,0.0,0.0,12,0,0,1,3529,...,0,3438,0,3438,0,14,0,13,7.070004,1024
1,31,0,2,0.0,0.0,14,9500,1,1,2140,...,0,5020,0,5020,0,11,0,0,1.429999,961
2,44,0,3,0.0,0.0,17,42500,1,1,4595,...,1,2815,0,2815,0,21,0,0,20.529997,1936
3,30,0,4,0.0,0.0,12,42500,1,1,3211,...,1,3786,0,3786,0,12,0,12,9.619998,900
4,64,0,5,0.0,0.0,14,2500,1,1,4052,...,1,2580,0,2580,0,44,0,33,2.750000,4096


## Спецификация и подгонка

In [6]:
# спецификация модели через формулу
mod = smf.ols(formula='sleep~totwrk+age+south+male+smsa+yngkid+marr+union+np.log(hrwage)', data=df)
# подгонка модели
res = mod.fit()

In [7]:
# Общий отчет
res.summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sleep   R-squared:                       0.123
Model:                            OLS   Adj. R-squared:                  0.108
No. Observations:                 532   F-statistic:                     8.117
Covariance Type:            nonrobust   Prob (F-statistic):           2.59e-11
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       3431.8036     96.309     35.633      0.000    3242.602    3621.005
totwrk            -0.1581      0.021     -7.576      0.000      -0.199      -0.117
age                2.4371      1.672      1.458      0.146      -0.847       5.721
south             78.0461     45.287      1.723      0.085     -10.921     167.014
male              36.4854     43.350      0.842      0.400     -48.677     121.648
smsa             -34.9646     38.232     -0.915      0.361    -110.072      40.143
yngkid            50.1365     54.790      0.915      0.361     -57.499     157.772
marr              54.0716     46.920      1.152      0.250     -38.104     146.247
union             27.0187     43.044      0.628      0.530     -57.542     111.580
np.log(hrwage)    -2.7268     32.202     -0.085      0.933     -65.989      60.536
==================================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.35e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Гипотеза

Тестируем значимость коэффициента $\beta_{age}$, т.е. гипотезу $H_0: \beta_{age} = 0$

## Уровень значимости
Пусть уровень значимости $\alpha=5\%=0.05$

Число наблюдений $n$, по которым была оценена модель

In [8]:
# Число наблюдений, по которым была оценена модель
res.nobs

532.0

Число регрессоров $k$, по которым была оценена модель

In [9]:
# Число регрессоров, по которым была оценена модель
res.df_model

9.0

Степени свободы для t-распределения $df=n-k-1$

In [10]:
# Степени свободы для t-распределения
res.df_resid

522.0

### Составляющие t-теста для коэффициентов

In [11]:
# стандартные ошибки s.e.(beta) для кажлого коэффциента с округленим
res.bse.round(3)

Intercept         96.309
totwrk             0.021
age                1.672
south             45.287
male              43.350
smsa              38.232
yngkid            54.790
marr              46.920
union             43.044
np.log(hrwage)    32.202
dtype: float64

In [12]:
# тестовые t-статистики для кажлого коэффциента с округленим
res.tvalues.round(3)

Intercept         35.633
totwrk            -7.576
age                1.458
south              1.723
male               0.842
smsa              -0.915
yngkid             0.915
marr               1.152
union              0.628
np.log(hrwage)    -0.085
dtype: float64

In [13]:
# P-значения для t-статистик с округленим
res.pvalues.round(4)

Intercept         0.0000
totwrk            0.0000
age               0.1455
south             0.0854
male              0.4004
smsa              0.3609
yngkid            0.3606
marr              0.2497
union             0.5305
np.log(hrwage)    0.9326
dtype: float64

In [14]:
# Составляющие t-теста для коэффициента(ов) в одной таблице
summary_params(res).round(3)

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
Intercept,3431.804,96.309,35.633,0.000,3242.602,3621.005
totwrk,-0.158,0.021,-7.576,0.000,-0.199,-0.117
age,2.437,1.672,1.458,0.146,-0.847,5.721
south,78.046,45.287,1.723,0.085,-10.921,167.014
male,36.485,43.350,0.842,0.400,-48.677,121.648
smsa,-34.965,38.232,-0.915,0.361,-110.072,40.143
yngkid,50.136,54.790,0.915,0.361,-57.499,157.772
marr,54.072,46.920,1.152,0.250,-38.104,146.247
union,27.019,43.044,0.628,0.530,-57.542,111.580
np.log(hrwage),-2.727,32.202,-0.085,0.933,-65.989,60.536


## Критическое значение t-распределения с уровнем значимости 5%=0.05

In [15]:
# уровнь значимости
sign_level = 0.05
# 5%-критическое значение t-распределения
t_cr = t.ppf(q=1-sign_level/2, df=res.df_resid).round(3)
t_cr

1.965

**<span style="color:#59afe1">ВЫВОД</span>**: На уровне значимости 5% значим коэффициент при `age`

## Какие коэффиценты значимы?

## Значимость всех коэффициентов

- проверяем значимость коэффициентов используя P-value (сравниваем самостоятельно или автоматизированно)

- проверяем значимость коэффициентов используя t_cr (сравниваем самостоятельно или автоматизированно)

**<span style="color:#59afe1">ОБЯЗАТЕЛЬНО ПЕЧАТАЕМ ВЫВОД</span>**

In [16]:
# проверим значимость коэффициентов используя P-value
df_sign = np.round(summary_params(res, alpha=sign_level), 3)
df_sign['significance'] = df_sign.apply(lambda x: 'Значим' if x['P>|t|']<sign_level else 'Незначим', axis=1)
df_sign

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975],significance
Intercept,3431.804,96.309,35.633,0.000,3242.602,3621.005,Значим
totwrk,-0.158,0.021,-7.576,0.000,-0.199,-0.117,Значим
age,2.437,1.672,1.458,0.146,-0.847,5.721,Незначим
south,78.046,45.287,1.723,0.085,-10.921,167.014,Незначим
male,36.485,43.350,0.842,0.400,-48.677,121.648,Незначим
smsa,-34.965,38.232,-0.915,0.361,-110.072,40.143,Незначим
yngkid,50.136,54.790,0.915,0.361,-57.499,157.772,Незначим
marr,54.072,46.920,1.152,0.250,-38.104,146.247,Незначим
union,27.019,43.044,0.628,0.530,-57.542,111.580,Незначим
np.log(hrwage),-2.727,32.202,-0.085,0.933,-65.989,60.536,Незначим


**<span style="color:#59afe1">ВЫВОД</span>**: На уровне значимости 5% значим коэффициент при `totwrk`

In [17]:
# проверим значимость коэффициентов используя t_cr
df_sign = np.round(summary_params(res, alpha=sign_level), 3)
df_sign['significance'] = df_sign.apply(lambda x: 'Значим' if np.abs(x['t'])>t_cr else 'Незначим', axis=1)
df_sign

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975],significance
Intercept,3431.804,96.309,35.633,0.000,3242.602,3621.005,Значим
totwrk,-0.158,0.021,-7.576,0.000,-0.199,-0.117,Значим
age,2.437,1.672,1.458,0.146,-0.847,5.721,Незначим
south,78.046,45.287,1.723,0.085,-10.921,167.014,Незначим
male,36.485,43.350,0.842,0.400,-48.677,121.648,Незначим
smsa,-34.965,38.232,-0.915,0.361,-110.072,40.143,Незначим
yngkid,50.136,54.790,0.915,0.361,-57.499,157.772,Незначим
marr,54.072,46.920,1.152,0.250,-38.104,146.247,Незначим
union,27.019,43.044,0.628,0.530,-57.542,111.580,Незначим
np.log(hrwage),-2.727,32.202,-0.085,0.933,-65.989,60.536,Незначим


**<span style="color:#59afe1">ВЫВОД</span>**: На уровне значимости 5% значим коэффициент при `totwrk`

## Значимость выбранных коэффициентов
Потестируем значимость коэффициентов $\beta_{totwrk}$ и $\beta_{male}$ на уровне значимости 5%

In [18]:
res.t_test('totwrk=0, male=0')

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0            -0.1581      0.021     -7.576      0.000      -0.199      -0.117
c1            36.4854     43.350      0.842      0.400     -48.677     121.648

## Критическое значение t-распределения с уровнем значимости 5%=0.05

In [19]:
# уровнь значимости
sign_level = 0.05
# 5%-критическое значение t-распределения
t_cr = t.ppf(q=1-sign_level/2, df=res.df_resid).round(3)
t_cr

1.965

**<span style="color:#59afe1">ВЫВОД</span>**: На уровне значимости 5% значим коэффициент при `totwrk`

---

# Общий t-тест

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

Тестируем $H_0:\beta=\theta$ ($\theta$ - заданное число)

Тестовая статистика $$t=\frac{\hat{\beta}-\theta}{s.e.(\beta)}$$

Критическое значение $t_{cr}=t_{df}(\alpha)$, где  $df=n-k-1$ --- степени свободы для t-распределения

$\color{purple}{\text{\bf Значимо ли коэффициент }\beta \text{ \bf отличается от числа } \theta?}$

**<span style="color: purple">Способ 1:</span>** используем $t_{cr}$

* Отвергаем гипотезу $H_0$ при $|t|>t_{cr}$, $\color{purple}{\text{\bf коэффициент }\beta \text{ \bf значимо отличается от числа } \theta?}$<br> 
* Не отвергаем гипотезу $H_0$ при $|t|<t_{cr}$, $\color{purple}{\text{\bf коэффициент }\beta \text{ \bf незначимо отличается от числа } \theta?}$<br> 

**<span style="color: purple">Способ 2:</span>** используем P-value

* Отвергаем гипотезу $H_0$  при $P<\alpha$, $\color{purple}{\text{\bf коэффициент }\beta \text{ \bf значимо отличается от числа } \theta?}$<br> 
* Не отвергаем гипотезу $H_0$ при $P>\alpha$, $\color{purple}{\text{\bf коэффициент }\beta \text{ \bf незначимо отличается от числа } \theta?}$<br> 

---

<i><b><span style="color: purple">Сравниваем самостоятельно</span> <br>

<span style="color:#59afe1">ОБЯЗАТЕЛЬНО ПЕЧАТАЕМ ВЫВОД</span></b>

---

</div>

Для набора данных `Labour` рассмотрим регрессию 

$$
\log(output) = \beta_0+\beta_1\log(capital) + \beta_2\log(labour)+error
$$

In [20]:
# импорт данных
df = pd.read_csv('Labour.csv')
df.head()

,capital,labour,output,wage
0,2.606563,184,9.250759,43.080307
1,1.323237,91,3.664310,27.780016
2,22.093692,426,28.781516,44.467748
3,10.737851,72,4.124642,39.734710
4,1.161365,46,2.890150,34.650709


## Спецификация и подгонка

In [21]:
# спецификация модели через формулу
mod = smf.ols(formula='np.log(output)~np.log(capital)+np.log(labour)', data=df)
# подгонка модели
res = mod.fit()

In [22]:
# Общий отчет
res.summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         np.log(output)   R-squared:                       0.838
Model:                            OLS   Adj. R-squared:                  0.837
No. Observations:                 569   F-statistic:                     1462.
Covariance Type:            nonrobust   Prob (F-statistic):          2.64e-224
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.7115      0.097    -17.697      0.000      -1.901      -1.522
np.log(capital)     0.2076      0.017     12.077      0.000       0.174       0.241
np.log(labour)      0.7148      0.023     30.890      0.000       0.669       0.760
===================================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Гипотеза
Тестируется гипотеза $H_0:\beta_{capital}=0.5$ 


## Уровень значимости
Пусть уровень значимости $\alpha=1\%=0.01$

## Результаты тестирования

In [23]:
# Составляющие общего t-теста для коэффициента(ов) в одной таблице
res.t_test('np.log(capital)=0.5')

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.2076      0.017    -17.014      0.000       0.174       0.241

## Критическое значение t-распределения с уровнем значимости 1%=0.01

In [24]:
# зададим уровнь значимости
sign_level = 0.01
# 1%-критическое значение t-распределения
t_cr = t.ppf(q=1-sign_level/2, df=res.df_resid).round(3)
t_cr

2.585

**<span style="color:#59afe1">ВЫВОД</span>**: На уровне значимости 1% коэффициент при `log(capital)` значимо отличается от числа 0.5